In [32]:
import numpy
from scipy.optimize import fsolve
import time
import sys

# Interview question

I was asked this problem during an interview. It is not really a brainteaser, it is more probability+optimisation.

A computer needs three components to work, but when I buy them, they have a probability $p_i$ to be delivered broken. I can buy no more than $N$ components overall. How do I distribute the three components in the overall set of $N$ purchases to maximise the probability to have a working computer?

Take $p_i$ to be the probability of a $i$-th component to be broken, $i=1,2,3$. I buy $n_i$ $i$-th components, so $N=n_1+n_2+n_3$.

The chance $P$ of having a working computer is:
$$P:=(1-p_1^{n_1})(1-p_2^{n_2})(1-p_3^{n_3})$$

Define the Lagrangian:
$$L(n_1,n_2,n_3,\lambda):=(1-p_1^{n_1})(1-p_2^{n_2})(1-p_3^{n_3})+\lambda(n_1+n_2+n_3-N)$$
so:
$$\frac{\partial L}{\partial \lambda}=0\Rightarrow N=n_1+n_2+n_3$$
$$\frac{\partial L}{\partial n_1}=0\Rightarrow (-n_1p_1^{n_1-1})(1-p_2^{n_2})(1-p_3^{n_3})+\lambda=0$$
and so on for $n_2$, $n_3$.
Multiply times $(1-p_1^{n_1})$ to both sides, find $P$, and simplify.
I get:
$$(-n_1p_1^{n_1-1})(1-p_2^{n_2})(1-p_3^{n_3})+\lambda=0\Rightarrow \frac{n_1p_1^{n_1-1}}{p_1^{n_1}-1}=-\frac \lambda P.$$

Note that $\lambda>0$, and so $K:=-\lambda/P<0$.
One can solve the problem this way:

0. set $K_0=0$. $K_j=K_{j-1}-\Delta$ for some $\Delta>0$ defining the accuracy of the method.
1. Solve the three equations ($i=1,2,3$) for $K_j$
$$\frac{n_ip_i^{n_i-1}}{p_i^{n_i}-1}=K_j,$$
and find if $\sum_i n_i < N$.
2. Find the combination of $\{n_i\}$ maximising the summation.

In [3]:
def RHS(p,n):
    return n*p**(n-1)/(p**n-1)

In [41]:
p1 = 0.1; p2 = 0.4; p3 = 0.6;
N = 20;

l = 0
v = 0
init_1 = 1
init_2 = 1
init_3 = 1

dl = 1e-5

start_time = time.time()
while (True):
    v += 1
    func1 = lambda n : RHS(p1,n) + l
    func2 = lambda n : RHS(p2,n) + l
    func3 = lambda n : RHS(p3,n) + l
    n1_solution = fsolve(func1, init_1)
    n2_solution = fsolve(func2, init_2)
    n3_solution = fsolve(func3, init_3)
    if (v % 1000 == 0):
        print("{2:7d}: lambda = {0:12.6e} - n1+n2+n3 = {1:8.4f}".format(l,n1_solution[0]+n2_solution[0]+n3_solution[0],v), end="\r")
        sys.stdout.flush()
    if ((n1_solution[0]+n2_solution[0]+n3_solution[0]) // 1 + 1 <= N):
        break
    l += dl
    if (2*n1_solution < N):
        init_1 = n1_solution
        init_2 = n2_solution
        init_3 = n3_solution
        
print(n1_solution[0] // 1, n2_solution[0] // 1, n3_solution[0] // 1, (n1_solution[0]+n2_solution[0]+n3_solution[0]) // 1)
print("--- %s seconds ---" % (time.time() - start_time))

2.0 6.0 11.0 19.0 4.999000e-02 - n1+n2+n3 =  20.6994
--- 1.9250140190124512 seconds ---
